In [1]:
import re

import pandas as pd

In [2]:
df = pd.read_csv("../data/output/rakuten_bandTshirts_prices/output.csv")

# テスト方針
- 変な商品が混じっていないか?
- 変な商品とは何か?
  - band_name列に格納されている値が、name列に含まれていないレコード
  - band_name列に対応する別名が、name列に含まれていないレコード
  - コラボ商品ではないレコード

In [48]:
# 表記揺れ対応用の辞書を作成
band_aliases = {
    "ローリングストーンズ": [
        "ローリング・ストーンズ",
        "rolling stones",
        "ROLLING STONE",
        "ローリングストーン",
    ],
    "ジミヘン": ["ジミ・ヘンドリックス", "jimi hendrix"],
    "METALLICA": ["メタリカ"],
    "Guns N’ Roses": [
        "Guns N' Roses",
        "ガンズ・アンド・ローゼズ",
        "Guns n Roses",
        "Guns N’Roses",
        "GUN'S & ROSES",
        "ガンズ＆ローゼス",
    ],
    "KISS": [],
    "アイアンメイデン": ["アイアン・メイデン", "アイアン メイデン", "IRON MAIDEN"],
    "スレイヤー": ["Slayer"],
    "ニルヴァーナ": ["ニルバーナ", "NIRVANA"],
    "カートコバーン": ["カート・コバーン", "Kurt Cobain", "カ-トコバーン"],
    "スリップノット": ["slipknot", "SlipKnot", "SLIPKNOT", "Slipknot"],
    "PINK FLOYD": ["PINKFLOYD", "pinkfloyd"],
    "JOURNEY": [],
}
# コラボブランド対応用
other_brand_list = [
    "COMMEdesGARCONS",
    "コムデギャルソン",
    "CHROME HEARTS",
    "クロムハーツ",
    "Supreme",
    "シュプリーム",
    "TMT",
    "Paul Smith",
    "glamb",
    "Adam et Rope",
    "HYSTERIC GLAMOUR",
    "XXX",
    "Sadistic Action",
    "MINEDENIM",
    "クリスチャンディオール",
    "HALFMAN",
]


# 関数を定義してバンド名を含むレコードを検出
def is_valid_record(band_name, product_name, band_aliases):
    if band_name not in band_aliases:
        return False
    aliases = [band_name] + band_aliases[band_name]
    pattern = r"(" + "|".join(re.escape(alias) for alias in aliases) + r")"

    # Check if product_name contains any of the other brands
    other_brand_pattern = (
        r"(" + "|".join(re.escape(brand) for brand in other_brand_list) + r")"
    )
    if re.search(other_brand_pattern, product_name, re.IGNORECASE):
        return False

    return bool(re.search(pattern, product_name, re.IGNORECASE))


# 不正なレコードの検出
df["is_valid"] = df.apply(
    lambda row: is_valid_record(row["band_name"], row["name"], band_aliases),
    axis=1,
)
invalid_records = df[~df["is_valid"]]
valid_records = df[df["is_valid"]]

In [46]:
# 不正なレコードは以下の通り
invalid_records

,name,price,url,band_name,is_valid
4,COMMEdesGARCONS コムデギャルソン xVersaceヴェルサーチェ【21年製】...,83600,https://hb.afl.rakuten.co.jp/hgc/g00s68be.brn1...,ローリングストーンズ,False
8,CHROME HEARTSクロムハーツローリングストーンズコラボTシャツ black,63800,https://hb.afl.rakuten.co.jp/hgc/g00pnkke.brn1...,ローリングストーンズ,False
14,The Rolling Stones × Paul Smith /ザ ローリング ストーンズ...,36080,https://hb.afl.rakuten.co.jp/hgc/g00siawe.brn1...,ローリングストーンズ,False
31,古着 80'S スクリーンスターズ SCREEN STARS THE ROLLING STO...,25960,https://hb.afl.rakuten.co.jp/hgc/g00qhzfe.brn1...,ローリングストーンズ,False
58,■【TMTティーエムティー】THE ROLLING STONES×TMT LSL TEE (...,19800,https://hb.afl.rakuten.co.jp/hgc/g00qk6ee.brn1...,ローリングストーンズ,False
...,...,...,...,...,...
1052,【中古】Supreme◆23SS/Kurt Cobain Tee/Tシャツ/XL/コットン/...,11990,https://hb.afl.rakuten.co.jp/hgc/g00r3cee.brn1...,カートコバーン,False
1053,【中古】HYSTERIC GLAMOUR NIRVANA LOVE BUZZ ／ KURT ...,11000,https://hb.afl.rakuten.co.jp/hgc/g00q49je.brn1...,カートコバーン,False
1056,HALFMANハーフマン×KURT COBAINカートコバーンPLAY THE GUITAR...,11000,https://hb.afl.rakuten.co.jp/hgc/g00pnkke.brn1...,カートコバーン,False
1060,Supreme Kurt Cobain Tee シュプリーム カートコバーン 半袖 Tシャツ...,10800,https://hb.afl.rakuten.co.jp/hgc/g00r4g4e.brn1...,カートコバーン,False


In [5]:
# 不正なレコードに対応するバンド名は以下の通り
invalid_records["band_name"].unique()

array(['KISS', 'スレイヤー', 'ニルヴァーナ', 'カートコバーン'], dtype=object)

In [6]:
# KISSは不明
invalid_records[invalid_records["band_name"] == "KISS"]["name"].tolist()

['【送料無料】 アクアスキュータム メンズ Tシャツ トップス T-shirts Black',
 'メゾンキツネ MAISON KITSUNE SS17W701-BK PERM TEE SHIRT JE SUIS ALLEE ブラック＋ホワイト Tシャツ レディース 半袖 サイズ トリコロール トルコカラー ロゴT ブランド おしゃれ 人気 20代 30代 40代 50代 母の日 クリスマス バレンタイン ホワイトデー',
 '【楽天スーパーSALE対象商品】メゾンキツネ MAISON KITSUNE SS17W701-BK PERM TEE SHIRT JE SUIS ALLEE ブラック＋ホワイト Tシャツ レディース 半袖 サイズ トリコロール トルコカラー ロゴT ブランド おしゃれ 人気 20代 30代 40代 50代',
 'サディスティックアクション SADISTIC ACTION レディース半袖Tシャツ アイコニック ICONIC COUTURE ブラック 新品 黒 アメリカ製',
 '【中古】X-LARGE\u3000×MONICHIKI S S WORKSHIRT 半袖カットソー 101223014007 ブラック サイズ：XL 【190524】（エクストララージ）']

In [7]:
# ニルヴァーナはカートコバーン
invalid_records[invalid_records["band_name"] == "ニルヴァーナ"]["name"].tolist()

['Supreme シュプリーム Tシャツ サイズ:L 23SS Kurt Cobain カート・コバーン フォト クルーネック 半袖 Tシャツ Kurt Cobain Tee ホワイト 白 トップス カットソー コラボ 【メンズ】【K4107】',
 'Supreme シュプリーム Tシャツ サイズ:S 23SS Kurt Cobain カート・コバーン フォト クルーネック Tシャツ Kurt Cobain Tee ペールブルー トップス カットソー 半袖 コラボ【メンズ】【中古】【美品】【K4107】']

In [8]:
# カートコバーンは不明
invalid_records[invalid_records["band_name"] == "カートコバーン"]["name"].tolist()

['デッドストック★90s Hanes Anatomical Chart Co. THE HEART 人体 臓器 心臓 コットンTシャツ 白 S【中古】']

In [9]:
# スレイヤーは~~ レイヤーで引っかかってしまっている
invalid_records[invalid_records["band_name"] == "スレイヤー"]["name"].tolist()

['【送料無料】MARNI / マルニ : 【レディース】T-SHIRT : Tシャツ 半袖 カットソー ホワイト ピンク ブラック 3枚セット 刺繍ロゴ 23SS 春夏 ウィメンズ トップス TEE デイリーユース レイヤードスタイル ミニマル : THJE0211X0【ANN】',
 '【抽選で全額ポイントバック企画！2人に1人が当選！！】【スーパーセール期間限定★クーポン配布中】モンクレール MONCLER レディーストップス バックファスナー Tシャツ 8C0000189AE8\u3000 \u3000フィットハウス',
 '【抽選で全額ポイントバック企画！2人に1人が当選！！】【スーパーセール期間限定★クーポン配布中】モンクレール MONCLER レディーストップス SS T-SHIRT 8C00006829HP\u3000 \u3000フィットハウス',
 "(取寄) スマートウール レディース クラシック オール-シーズン メリノ ベース レイヤー ロング-スローブ - ウィメンズ Smartwool women Classic All-Season Merino Base Layer Long-Sleeve - Women's Black",
 '(取寄) ボビ ロサンゼルス レディース スプリットネック リブ ミックス レイヤード Tシャツ bobi Los Angeles women bobi Los Angeles Split-Neck Long Sleeve Rib Mix Layered Tee Black',
 '(取寄) ボビ ロサンゼルス レディース スプリットネック リブ ミックス レイヤード Tシャツ bobi Los Angeles women bobi Los Angeles Split-Neck Long Sleeve Rib Mix Layered Tee Olive Tree',
 "(取寄) スマートウール レディース クラシック オール-シーズン メリノ ベース レイヤー ロング-スローブ - ウィメンズ Smartwool women Classic All-Season Merino Base Layer Long-Sleeve - Women's Black",
 "(取寄) スマートウール レディース クラシ

In [10]:
# カートコバーンはニルヴァーナに統一する
invalid_records[invalid_records["band_name"] == "ニルヴァーナ"]

,name,price,url,band_name,is_valid
916,Supreme シュプリーム Tシャツ サイズ:L 23SS Kurt Cobain カート...,22450,https://hb.afl.rakuten.co.jp/hgc/g00s4pxe.brn1...,ニルヴァーナ,False
927,Supreme シュプリーム Tシャツ サイズ:S 23SS Kurt Cobain カート...,16400,https://hb.afl.rakuten.co.jp/hgc/g00s4pxe.brn1...,ニルヴァーナ,False


## コラボ

In [11]:
# nameにハイブランドが含まれていたら、それらを除外する

In [12]:
valid_records.shape

(1142, 5)

In [13]:
other_brand_list = [
    "COMMEdesGARCONS",
    "コムデギャルソン",
    "CHROME HEARTS",
    "クロムハーツ",
    "Supreme",
    "シュプリーム",
]

In [14]:
# 関数を定義してブランド名を含まないレコードを検出
def is_valid_record_other_brand(other_brand_list, product_name):
    return any(other_brand not in product_name for other_brand in other_brand_list)

In [15]:
valid_records["is_valid_other_brand"] = valid_records.apply(
    lambda row: is_valid_record_other_brand(other_brand_list, row["name"]),
    axis=1,
)

/var/folders/c7/_dn3d3y51mg8sp2nzy21k0q80000gn/T/ipykernel_88809/269083910.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_records["is_valid_other_brand"] = valid_records.apply(


In [18]:
valid_records[valid_records["is_valid_other_brand"]]

,name,price,url,band_name,is_valid,is_valid_other_brand
0,"90's ROLLING STONES USA製 ""3 Ball Charlie"" ロックバ...",107800,https://hb.afl.rakuten.co.jp/hgc/g00taeee.brn1...,ローリングストーンズ,True,True
1,"【中古】70's The Rolling Stones/ローリング・ストーンズ ""Tour ...",87780,https://hb.afl.rakuten.co.jp/hgc/g00tzn6e.brn1...,ローリングストーンズ,True,True
2,DEADSTOCK 90s USA製 ヴィンテージ ローリングストーンズ The Rolli...,84800,https://hb.afl.rakuten.co.jp/hgc/g00rgike.brn1...,ローリングストーンズ,True,True
3,DEADSTOCK 90s USA製 ヴィンテージ ローリングストーンズ The Rolli...,84800,https://hb.afl.rakuten.co.jp/hgc/g00rgike.brn1...,ローリングストーンズ,True,True
4,COMMEdesGARCONS コムデギャルソン xVersaceヴェルサーチェ【21年製】...,83600,https://hb.afl.rakuten.co.jp/hgc/g00s68be.brn1...,ローリングストーンズ,True,True
...,...,...,...,...,...,...
1238,【中古】80s/journey/ラグラン7分袖カットソー/GRY/プリント【メンズウェア】,10890,https://hb.afl.rakuten.co.jp/hgc/g00r3cee.brn1...,JOURNEY,True,True
1239,【中古】Journey_1982s_MUSIC-T Tシャツ/M/コットン/GRY【メンズウェア】,10890,https://hb.afl.rakuten.co.jp/hgc/g00r3cee.brn1...,JOURNEY,True,True
1240,"(取寄) アイベックス ジャーニー T-シャツ - メリノ ウール, ショート スリーブ I...",10390,https://hb.afl.rakuten.co.jp/hgc/g00qwfoe.brn1...,JOURNEY,True,True
1241,"(取寄) アイベックス ジャーニー シャツ - メリノ ウール, ショート スリーブ Ibe...",10390,https://hb.afl.rakuten.co.jp/hgc/g00rhtxe.brn1...,JOURNEY,True,True


# valid_records目検チェック

In [64]:
# 【】で囲まれた部分を削除する正規表現パターン
pattern = r"【[^】]*】"
cleaned_v1 = [re.sub(pattern, "", v) for v in valid_records["name"]]

# 特定の単語を削除する正規表現パターン
words_to_remove = [
    "半袖",
    "音楽",
    "ミュージック",
    "バンT",
    "Tee",
    "半袖Tシャツ",
    "古着",
    "★",
    "メンズ",
    "ロックバンドプリントTシャツ",
    "表記XLサイズ",
    "表記Mサイズ",
    "XL",
    "バンドTシャツ",
    "Tシャツ",
    "カットソー",
    "トップス",
    "クルーネック"
]
pattern_words = (
    r"\b(?:" + "|".join(re.escape(word) for word in words_to_remove) + r")\b"
)

cleaned_v2 = [re.sub(pattern_words, "", v) for v in cleaned_v1]

# 複数の空白を1つの空白に変換
[re.sub(r"\s+", " ", v).strip() for v in cleaned_v2]

['90\'s ROLLING STONES USA製 "3 Ball Charlie" 90年代 90s ローリングストーンズ',
 '70\'s The Rolling Stones/ローリング・ストーンズ "Tour Of The Americas 75" 230509H',
 'DEADSTOCK 90s USA製 ヴィンテージ ローリングストーンズ The Rolling Stones BROCKUM ★',
 'DEADSTOCK 90s USA製 ヴィンテージ ローリングストーンズ The Rolling Stones BROCKUM ★ ホワイト',
 "80's vintage MICK JAGGER THE ROLLING STONES JAPAN TOUR 88 両面プリント",
 'Rolling Stones / Keith Richards Start Me Up ザ・ローリング・ストーンズ キース・リチャーズ Vintage T-shirt ヴィンテージ',
 '90s/94年製/The Rolling Stones/Voodoo Lounge/WHT/プリント',
 '90s/BROCKUM//L/コットン/BLK/THE Rolling Stones',
 'VINTAGE(ヴィンテージ) サイズ: 90S VINTAGE ヴィンテージ The Rolling Stones World Tour Voodoo Lounge ローリングストーンズ スカル プリント半袖Tシャツ ブラック',
 'BAND-T 90s ローリングストーンズ Rolling Stones ワールドツアー 94/95 Tour T-Shirt ヴィンテージTシャツ US古着 プリント ブラック LLサイズ 101MT-2384',
 '希少 90s USA製 The Rolling Stones スタッフ プリント L / 90年代 アメリカ製 ローリングストーンズ バンド アメリカ古着 USED ユーズド 中古 VINTAGE US古着 アメカジ',
 'THE ROLLING STONES///コットン/BLK',
 "THE BROCKUM COLLECTION BROCKUM 89'S ROLLING STONES THE NORTH AMERICA